# Google colab initialization

For Google colab uncomment these lines and run them to access your drive or try the second way (not tested)

In [1]:
# from google.colab import drive
# drive.mount('/content/drive')
# import sys
#
# sys.path.insert(1, r'/content/drive/My Drive/')

In [2]:
# Other try
# !git clone https://github.com/Alexanderstaehle/OM_project

In [3]:
# import sys
# sys.path.append("OM_project")

In [4]:
import seaborn as sns
import tensorflow as tf
from tensorflow import keras

from utils import ml_utils, visualization, data_loading, tf_models

2022-02-19 00:58:25.942145: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-02-19 00:58:25.942161: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [5]:
data_loading.initialize_env()
sns.set_theme()
color_map = sns.color_palette(as_cmap=True)
ml_utils.check_tpu_gpu()

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:CPU:0',)
Number of accelerators:  1


2022-02-19 00:58:27.188485: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-02-19 00:58:27.188876: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-02-19 00:58:27.188927: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory
2022-02-19 00:58:27.188972: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared object file: No such file or directory
2022-02-19 00:58:27.189017: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Co

# Different batch sizes with fixed learning rate

In [6]:
models_dict_fixed = {}
batch_sizes = [32, 64, 128, 256, 512, 1024]
# batch_sizes = [32, 64, 128]
lr = 0.001
training_epochs = 10

In [7]:
# test of SAM model 

batch_size = 32

train, validation = data_loading.load_batched_and_resized_dataset(dataset_name='MNIST', batch_size=batch_size,
                                                                 img_size=32)

#base_model = tf_models.build_simple_dense_model(train)
base_model = tf_models.build_simple_cnn(train)
train_callbacks = [
        tf.keras.callbacks.EarlyStopping(
            monitor="val_loss", patience=10,
            restore_best_weights=True
        ),
        # tf.keras.callbacks.ReduceLROnPlateau(
        #     monitor="val_loss", factor=0.5,
        #     patience=3, verbose=1
        # )
    ]
model = tf_models.SAMModel(base_model, adaptive = True, rho = 2.0)    

optimizer = keras.optimizers.SGD(learning_rate=lr)
model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
models_dict_fixed[batch_size] = ml_utils.train_model(model, train, validation, epochs=1,
                                                         extra_callbacks=train_callbacks, verbose=1)

1500/1500 [==============================] - 30s 20ms/step - loss: 2.2501 - accuracy: 0.2073 - val_loss: 2.1171 - val_accuracy: 0.4559


In [8]:
base_model.set_weights(model.get_weights())
base_model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [10]:
base_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 32, 32, 32)        320       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 16, 16, 32)       0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 16, 16, 32)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 16, 16, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 8, 8, 64)         0         
 2D)                                                             
                                                                 
 dropout_1 (Dropout)         (None, 8, 8, 64)          0

In [18]:
for x, y in train:
    print(base_model(x))
    break

tf.Tensor(
[[0.11003389 0.06785528 0.10996836 0.11130752 0.10856888 0.08450007
  0.10663995 0.10150438 0.09082206 0.10879955]
 [0.09318502 0.08664616 0.10416766 0.1336858  0.08721875 0.092296
  0.08961476 0.09513535 0.11861256 0.09943791]
 [0.10435594 0.08293848 0.14660339 0.15378879 0.07196042 0.08983143
  0.10219627 0.0733836  0.09962153 0.07532008]
 [0.09844474 0.11450472 0.10852458 0.1199836  0.09081478 0.08433477
  0.08713105 0.09418996 0.11577658 0.0862952 ]
 [0.0863365  0.13019386 0.10117725 0.11790382 0.0906997  0.09128471
  0.09408189 0.08815446 0.11025095 0.08991682]
 [0.1378163  0.08292704 0.09902631 0.11484574 0.08654949 0.09294702
  0.08524287 0.10161629 0.1180331  0.08099589]
 [0.09498664 0.09423433 0.08920905 0.11889412 0.09490675 0.09430961
  0.08557387 0.11066682 0.10824472 0.10897408]
 [0.11800514 0.08216178 0.10369252 0.10788831 0.09338895 0.08644692
  0.09403294 0.10457719 0.11442316 0.09538311]
 [0.11041544 0.09220587 0.10740732 0.10492489 0.08924282 0.0953768
  0.

In [9]:
visualization.get_sharpness(base_model, train)

1500/1500 [==============================] - 5s 3ms/step - loss: 2.1159 - accuracy: 0.4586


ValueError: `logits` and `labels` must have the same shape, received ((32, 10) vs (32,)).

In [ ]:
for batch_size in batch_sizes:
    # Read training data
    train, validation = data_loading.load_batched_and_resized_dataset(dataset_name='MNIST', batch_size=batch_size,
                                                                      img_size=32)
    model = tf_models.build_simple_dense_model(train)
    train_callbacks = [
        tf.keras.callbacks.EarlyStopping(
            monitor="val_loss", patience=10,
            restore_best_weights=True
        ),
        # tf.keras.callbacks.ReduceLROnPlateau(
        #     monitor="val_loss", factor=0.5,
        #     patience=3, verbose=1
        # )
    ]
    optimizer = keras.optimizers.Adam(learning_rate=lr)
    # Need sparse categorical crossentropy since our labels are in form of integers not vectors
    model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    models_dict_fixed[batch_size] = ml_utils.train_model(model, train, validation, epochs=10,
                                                         extra_callbacks=train_callbacks, verbose=1)

In [ ]:
ml_utils.save_model_state(models_dict_fixed, 'model_fixed_lr_diff_bs')

In [ ]:
visualization.plot_loss_by_param(models_dict_fixed, 'batch size with fixed learning rate', 'fixed_lr_diff_bs')


# Different batch sizes with linear increasing learning rate

In [ ]:
learning_rates = [0.001, 0.002, 0.004, 0.008, 0.016, 0.032]